In [33]:
import numpy as np
import pandas as pd
import datetime
import math


In [49]:
dam = pd.read_csv('dam.csv')
dam

,hourlyid,market,tradedate,hour,price,volume
0,26345,1,2016-01-01,0,40.10,477.5
1,26346,1,2016-01-01,1,40.08,486.6
2,26347,1,2016-01-01,2,40.06,495.6
3,26348,1,2016-01-01,3,40.08,496.6
4,26349,1,2016-01-01,4,40.07,502.9
...,...,...,...,...,...,...
46723,73029,1,2021-04-29,4,39.97,126.1
46724,73037,1,2021-04-29,12,68.06,71.0
46725,73049,1,2021-04-30,0,44.04,59.6
46726,73057,1,2021-04-30,8,190.00,72.0


In [50]:
dam['date']=pd.to_datetime(dam['tradedate'],format="%Y/%m/%d")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam=dam.drop(['hourlyid','tradedate','date'],axis=1)
dam

,market,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,
2016-01-01,1,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0
2016-01-01,1,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1
2016-01-01,1,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2
2016-01-01,1,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3
2016-01-01,1,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4
...,...,...,...,...,...,...,...,...,...,...
2021-04-29,1,4,39.97,126.1,2021,4,29,3,2021.04,weekday_4
2021-04-29,1,12,68.06,71.0,2021,4,29,3,2021.04,weekday_12
2021-04-30,1,0,44.04,59.6,2021,4,30,4,2021.04,weekday_0


In [51]:
dam.loc[dam['hour']<12].groupby(['day','month','year']).max()['price']


day  month  year
1    1      2016     48.03
            2017     33.47
            2018     34.16
            2019     27.26
            2020     80.95
                     ...  
31   12     2016     53.28
            2017     31.57
            2018     56.80
            2019     88.52
            2020    102.37
Name: price, Length: 1947, dtype: float64

In [52]:
dam['value']=dam['price']*dam['volume']
daily=dam.groupby(['year','month','day']).sum().drop(['hour','day_of_week','month_no','market'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=dam[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=dam[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=dam[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=dam[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=dam[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=dam[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=dam.loc[dam['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=dam.loc[dam['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=dam.loc[(dam['hour']>=12) & (dam['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 4     26   2092.9  185605.239        88.683281     299.33      39.94   
           27   2048.2  168261.497        82.150912     299.09      39.96   
           28   2130.2  177543.993        83.346161     299.10      39.90   
           29   2134.0  178697.984        83.738512     296.82      39.79   
           30   2198.9  187057.475        85.068659     295.98      38.97   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2021 4     26   102.215000      156.1       40.0   87.204167   
           27    94.468333      126.2       54.7   85.341667   
           28   100.343333      145.1       54.0   88.758333   
           29    98.742500      225.5       60.0   88.916667   
           30    98.218750      201.9       59.5   91.620833   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2016 1     1                48.03              60.08         45.8275  
           2                56.05             118.49         50.2275  
           3                56.01             118.47         50.6050  
           4                80.02             177.66         80.0200  
           5               148.40             181.40         80.0500  
...                           ...                ...             ...  
2021 4     26              201.07             299.33         70.0450  
           27              201.03             299.09         70.0200  
           28              198.58             299.10         70.0125  
           29              195.01             296.82         68.0600  
           30              190.00             295.98         67.9825  

[1947 rows x 12 columns]

In [53]:
dam=dam.reset_index()#.set_index(['year','month','day'])
dam

,date,market,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref,value
0,2016-01-01,1,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0,19147.750
1,2016-01-01,1,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1,19502.928
2,2016-01-01,1,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2,19853.736
3,2016-01-01,1,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3,19903.728
4,2016-01-01,1,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4,20151.203
...,...,...,...,...,...,...,...,...,...,...,...,...
46723,2021-04-29,1,4,39.97,126.1,2021,4,29,3,2021.04,weekday_4,5040.217
46724,2021-04-29,1,12,68.06,71.0,2021,4,29,3,2021.04,weekday_12,4832.260
46725,2021-04-30,1,0,44.04,59.6,2021,4,30,4,2021.04,weekday_0,2624.784
46726,2021-04-30,1,8,190.00,72.0,2021,4,30,4,2021.04,weekday_8,13680.000


In [54]:

#dam['price_min']=daily['price_min']
#dam['price_max']=daily['price_max']
#dam['price_mean']=daily['price_mean']
#dam['price_standard']=daily['price_standard']

#dam['price_morning_max']=daily['price_morning_max']
#dam['price_evening_max']=daily['price_evening_max']


dam['cat']='off'
dam.loc[(dam['day_of_week']<5) & (dam['hour']>5) & (dam['hour']<10),'cat']='morning'
dam.loc[(dam['day_of_week']<5) & (dam['hour']>9) & (dam['hour']<17),'cat']='standard'

dam.loc[(dam['day_of_week']<5) & (dam['hour']==17) & (dam['month']>=6) & (dam['month']<=8),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==17) & ((dam['month']<6) | (dam['month']>8)),'cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==18) & (dam['month']<=3),'cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==18) & (dam['month']>3),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==19),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==20) & (dam['month']<=3),'cat']='evening'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==20) & (dam['month']>3),'cat']='standard'
dam.loc[(dam['day_of_week']<5) & (dam['hour']==21),'cat']='standard'

dam.loc[(dam['day_of_week']==5) & (dam['hour']>5) & (dam['hour']<10),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==17) & (dam['month']>=6) & (dam['month']<=8),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==18) & (dam['month']>3),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==19),'cat']='standard'
dam.loc[(dam['day_of_week']==5) & (dam['hour']==20) & (dam['month']<=3),'cat']='standard'






#dam['cat']='standard'
#dam.loc[(dam['hour']<11) & (dam['price_morning_max'] > dam['price_standard'] + (dam['price_standard'] - dam['price_min'])) & (dam['price'] > dam['price_morning_max']-(dam['price_mean'] * 0.2)),'cat']='morning'
#dam.loc[(dam['hour']>=11) & (dam['price_evening_max'] > dam['price_standard'] + (dam['price_standard'] - dam['price_min'])) & (dam['price'] > dam['price_evening_max']-(dam['price_mean'] * 0.2)),'cat']='evening'

#dam.loc[dam['price'] < dam['price_min']+(dam['price_mean'] * 0.2),'cat']='off'

#dam.loc[dam['price_max'] - dam['price_min']<30,'cat']='off'
dam

,date,market,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref,value,cat
0,2016-01-01,1,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0,19147.750,off
1,2016-01-01,1,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1,19502.928,off
2,2016-01-01,1,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2,19853.736,off
3,2016-01-01,1,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3,19903.728,off
4,2016-01-01,1,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4,20151.203,off
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46723,2021-04-29,1,4,39.97,126.1,2021,4,29,3,2021.04,weekday_4,5040.217,off
46724,2021-04-29,1,12,68.06,71.0,2021,4,29,3,2021.04,weekday_12,4832.260,standard
46725,2021-04-30,1,0,44.04,59.6,2021,4,30,4,2021.04,weekday_0,2624.784,off
46726,2021-04-30,1,8,190.00,72.0,2021,4,30,4,2021.04,weekday_8,13680.000,morning


In [55]:
daily['off_price_mean']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

#daily=daily.drop(['price_min','price_max','price_morning_max','price_evening_max'], axis=1)

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 4     26   2092.9  185605.239        88.683281     299.33      39.94   
           27   2048.2  168261.497        82.150912     299.09      39.96   
           28   2130.2  177543.993        83.346161     299.10      39.90   
           29   2134.0  178697.984        83.738512     296.82      39.79   
           30   2198.9  187057.475        85.068659     295.98      38.97   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2021 4     26   102.215000      156.1       40.0   87.204167   
           27    94.468333      126.2       54.7   85.341667   
           28   100.343333      145.1       54.0   88.758333   
           29    98.742500      225.5       60.0   88.916667   
           30    98.218750      201.9       59.5   91.620833   

                price_morning_max  ...  morning_price_mean  \
year month day                     ...                       
2016 1     1                48.03  ...             44.0600   
           2                56.05  ...                 NaN   
           3                56.01  ...                 NaN   
           4                80.02  ...             75.7400   
           5               148.40  ...             80.0350   
...                           ...  ...                 ...   
2021 4     26              201.07  ...            145.3000   
           27              201.03  ...            145.2450   
           28              198.58  ...            139.4850   
           29              195.01  ...            137.7125   
           30              190.00  ...            135.2125   

                evening_price_mean  off_hour_count  standard_hour_count  \
year month day                                                            
2016 1     1                60.050               8                 10.0   
           2                   NaN              18                  6.0   
           3                   NaN              24                  NaN   
           4               128.860               8                 10.0   
           5               161.235               8                 10.0   
...                            ...             ...                  ...   
2021 4     26              299.215               8                 10.0   
           27              299.085               8                 10.0   
           28              299.085               8                 10.0   
           29              296.805               8                 10.0   
           30              295.975               8                 10.0   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   4.0                 2.0      3851.5   
           2                   NaN                 NaN      5901.4   
           3                   NaN                 NaN      7198.7   
           4                   4.0              

In [56]:
daily.loc[2016,2,8]

volume                   3114.100000
value                  195967.843000
price_weight_av            62.929207
price_max                 171.760000
price_min                  50.360000
price_mean                 96.732917
power_max                 398.400000
power_min                   0.000000
power_mean                129.754167
price_morning_max         171.760000
price_evening_max         145.020000
price_standard            103.080000
off_price_mean             50.618750
standard_price_mean       109.710000
morning_price_mean        138.607500
evening_price_mean        132.555000
off_hour_count              8.000000
standard_hour_count        10.000000
morning_hour_count          4.000000
evening_hour_count          2.000000
off_volume               2333.400000
standard_volume           600.000000
morning_volume            168.800000
evening_volume             11.900000
Name: (2016, 2, 8), dtype: float64

In [57]:
daily.to_csv('dam_daily.csv')

In [58]:
dam=dam.drop(['hour_ref','month_no'],1)

In [59]:
dam.to_csv('dam_out.csv')

In [60]:
dam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46728 entries, 0 to 46727
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         46728 non-null  datetime64[ns]
 1   market       46728 non-null  int64         
 2   hour         46728 non-null  int64         
 3   price        46728 non-null  float64       
 4   volume       46728 non-null  float64       
 5   year         46728 non-null  int64         
 6   month        46728 non-null  int64         
 7   day          46728 non-null  int64         
 8   day_of_week  46728 non-null  int64         
 9   value        46728 non-null  float64       
 10  cat          46728 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(6), object(1)
memory usage: 3.9+ MB


In [61]:
dam_flat=dam.reset_index().set_index(['date'])
dam_slim=dam_flat.loc[(dam_flat['year']<2021)][['hour','price','volume','cat']]
dam_slim

,hour,price,volume,cat
date,,,,
2016-01-01,0,40.10,477.5,off
2016-01-01,1,40.08,486.6,off
2016-01-01,2,40.06,495.6,off
2016-01-01,3,40.08,496.6,off
2016-01-01,4,40.07,502.9,off
...,...,...,...,...
2020-12-30,7,102.42,91.0,morning
2020-12-30,16,60.03,84.0,standard
2020-12-31,1,50.15,144.0,off


In [62]:
dam_slim.to_csv('dam_slim.csv')